# Cryptopals Challenge Set 5

https://cryptopals.com/sets/5

## 33. Implement Diffie-Hellman

https://cryptopals.com/sets/5/challenges/33

https://en.wikipedia.org/wiki/Diffie–Hellman_key_exchange

In [1]:
import random

# public parameters
p = 37
g = 5

# secret keys
a = random.randint(0,p-1)
b = random.randint(0,p-1)

# public keys
A = pow(g,a,p)
B = pow(g,b,p)

# common secret key
print(pow(B,a,p)==pow(A,b,p))

True


In [9]:
p = int(
'ffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024'
'e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd'
'3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec'
'6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f'
'24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361'
'c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552'
'bb9ed529077096966d670c354e4abc9804f1746c08ca237327fff'
'fffffffffffff', 16)

g = 2

a = random.randint(0,p-1)
b = random.randint(0,p-1)

A = pow(g,a,p)
B = pow(g,b,p)

print(pow(B,a,p)==pow(A,b,p))

True


## 34. Implement a MITM key-fixing attack on Diffie-Hellman with parameter injection

https://cryptopals.com/sets/5/challenges/34

In [139]:
import os
import random
from Cryptodome.Cipher import AES
from hashlib import sha1 as SHA1
import math
from cryptopals import pkcs7_pad, pkcs7_strip

class DHA:
    def __init__(self, 
                 p = int(
        'ffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024'
        'e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd'
        '3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec'
        '6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f'
        '24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361'
        'c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552'
        'bb9ed529077096966d670c354e4abc9804f1746c08ca237327fff'
        'fffffffffffff', 16), 
                g = 2):
        self.p = p
        self.g = g
        self.a = random.randint(0,self.p-1)
        self.A = pow(self.g, self.a, self.p)
        self.B = None
        self.key = None

    def send_params(self):
        return {"p": self.p, "g": self.g, "A": self.A}

    def generate_key(self):
        if self.B:
            s = pow(self.B,self.a,self.p)
            max_len = math.ceil(math.log2(self.p) / 8)
            s_bytes = s.to_bytes(max_len, 'big')
            self.key = SHA1(s_bytes).digest()[0:16]
            return True
        else:
            print("Cannot generate key, please complete session initialition before!")
            return False

    def send_message(self,message):
        if not self.key:
            if not self.generate_key():
                return None
        iv = os.urandom(16)
        aes_cbc = AES.new(self.key,AES.MODE_CBC,iv)
        cipher = aes_cbc.encrypt(pkcs7_pad(message))
        return {"cipher": cipher, "iv": iv}

    def receive_message(self,message,verbose=True):
        if not self.key:
            if not self.generate_key():
                return None
        aes_cbc = AES.new(self.key,AES.MODE_CBC,message["iv"])
        plaintext = pkcs7_strip(aes_cbc.decrypt(message["cipher"]))
        if verbose:
            print(f"A received '{plaintext.decode()}'")
        iv = os.urandom(16)
        aes_cbc = AES.new(self.key,AES.MODE_CBC,iv)
        cipher = aes_cbc.encrypt(pkcs7_pad(plaintext))
        return {"cipher": cipher, "iv": iv}

class DHB:
    def __init__(self):
        self.p = None
        self.g = None
        self.b = None
        self.B = None
        self.A = None
        self.key = None

    def get_params(self, params):
        self.p = params["p"]
        self.g = params["g"]
        self.A = params["A"]
        self.b = random.randint(0,self.p-1) 
        self.B = pow(self.g, self.b, self.p)
        return self.B

    def generate_key(self):
        if self.A:
            s = pow(self.A,self.b,self.p)
            max_len = math.ceil(math.log2(self.p) / 8)
            s_bytes = s.to_bytes(max_len, 'big')
            self.key = SHA1(s_bytes).digest()[0:16]
            return True
        else:
            print("Cannot generate key, please complete session initialition before!")
            return False

    def receive_message(self,message,verbose=True):
        if not self.key:
            if not self.generate_key():
                return None
        aes_cbc = AES.new(self.key,AES.MODE_CBC,message["iv"])
        plaintext = pkcs7_strip(aes_cbc.decrypt(message["cipher"]))
        if verbose:
            print(f"B received '{plaintext.decode()}'")
        iv = os.urandom(16)
        aes_cbc = AES.new(self.key,AES.MODE_CBC,iv)
        cipher = aes_cbc.encrypt(pkcs7_pad(plaintext))
        return {"cipher": cipher, "iv": iv}

In [140]:
A = DHA()
B = DHB()

# initialize session
A.B = B.get_params( A.send_params() )

# exchange message
_ = A.receive_message( B.receive_message( A.send_message(b"test message") ) )

B received 'test message'
A received 'test message'


### Attack 
> "A" and "B" in the protocol --- the public keys, over the wire --- have been swapped out with "p". 

`A = B = p`

`s = pow(p,a,p) = pow(p,b,p)) = 0` 

In [153]:
class MITM:
    def __init__(self):
        self.p = None
        self.g = None
        self.b = None
        self.B = None
        self.A = None
        self.key = None

    def relay_params(self, params): # Receive session parameters from A
        self.p = params["p"]
        self.g = params["g"]
        self.A = params["A"]
        self.b = random.randint(0,self.p-1) 
        self.B = pow(self.g, self.b, self.p)
        return {"p": self.p, "g": self.g, "A": self.p} # Send "p", "g", "p" to B

    def relay_B(self, B):
        self.B = B
        return self.p

    def generate_key(self):
        if self.p:
            s = 0
            max_len = math.ceil(math.log2(self.p) / 8)
            s_bytes = s.to_bytes(max_len, 'big')
            self.key = SHA1(s_bytes).digest()[0:16]
            return True
        else:
            print("Cannot generate key, please complete session initialition before!")
            return False
    
    def relay_message(self,message,verbose=True):
        if not self.key:
            if not self.generate_key():
                return None
        aes_cbc = AES.new(self.key,AES.MODE_CBC,message["iv"])
        plaintext = pkcs7_strip(aes_cbc.decrypt(message["cipher"]))
        if verbose:
            print(f"M received '{plaintext.decode()}'")
        return message

In [154]:
A = DHA()
B = DHB()
M = MITM()

A.B = M.relay_B( B.get_params( M.relay_params( A.send_params() )))

_ = A.receive_message( M.relay_message( B.receive_message( M.relay_message( A.send_message(b"test message") ))))

M received 'test message'
B received 'test message'
M received 'test message'
A received 'test message'
